# Когортный анализ

In [1]:
import pandas as pd

## Задача
По данным заказов в интернет-магазине требуется понять, сколько заказов и денег клиенты приносят в течение года после первой покупки.

In [2]:
orders = pd.read_csv('EDA_data/orders.csv')
orders.head()

,id,order_date,ship_mode,customer_id,sales
0,100006,2014-09-07,Standard,DK-13375,377.970
1,100090,2014-07-08,Standard,EB-13705,699.192
2,100293,2014-03-14,Standard,NF-18475,91.056
3,100328,2014-01-28,Standard,JC-15340,3.928
4,100363,2014-04-08,Standard,JM-15655,21.376


In [3]:
orders['order_date'] = pd.to_datetime(orders['order_date'])

## Считаем покупки и выручку

In [4]:
orders['sales'].count()

5009

In [5]:
orders['sales'].sum()

2297200.8603000003

In [6]:
orders.groupby('customer_id')['sales'].agg(['sum', 'count'])

,sum,count
customer_id,,
AA-10315,5563.560,5
AA-10375,1056.390,9
AA-10480,1790.512,4
AA-10645,5086.935,6
AB-10015,886.156,3
...,...,...
XP-21865,2374.658,11
YC-21895,5454.350,5
YS-21880,6720.444,8


## Считаем дату первой покупки

In [7]:
first_orders = orders.groupby('customer_id')['order_date'].agg(first_order='min')
first_orders.head()

,first_order
customer_id,
AA-10315,2014-03-31
AA-10375,2014-04-21
AA-10480,2014-05-04
AA-10645,2014-06-22
AB-10015,2014-02-18


## Строим когорты

In [8]:
orders_merged = orders.merge(first_orders, how='inner', left_on='customer_id', right_index=True)
orders_merged.head()

,id,order_date,ship_mode,customer_id,sales,first_order
0,100006,2014-09-07,Standard,DK-13375,377.970,2014-09-07
1219,131884,2015-12-06,Same Day,DK-13375,594.002,2014-09-07
1398,145065,2015-12-12,First,DK-13375,32.308,2014-09-07
3463,133046,2017-07-27,Second,DK-13375,297.990,2014-09-07
4102,165099,2017-12-11,First,DK-13375,1.392,2014-09-07


In [9]:
orders_merged.groupby('first_order')['sales'].agg(['sum', 'count'])

,sum,count
first_order,,
2014-01-03,1050.636,9
2014-01-04,1056.858,5
2014-01-05,1428.231,7
2014-01-06,14287.557,22
2014-01-07,1055.981,4
...,...,...
2017-07-08,1058.108,1
2017-09-15,1038.260,1
2017-10-21,729.648,1


Видим, что клиенты от 3 января 2014 года, всего сделали 9 заказов на $1050.6. Посмотрим, когда были эти заказы.

In [10]:
orders_merged.groupby(['first_order','order_date'])['sales'].agg(['sum', 'count'])


sum  count
first_order order_date                 
2014-01-03  2014-01-03    16.448      1
            2014-11-12   153.112      1
            2015-04-18   209.550      1
            2015-11-24   383.610      1
            2016-05-15     7.764      1
...                          ...    ...
2017-09-15  2017-09-15  1038.260      1
2017-10-21  2017-10-21   729.648      1
2017-10-23  2017-10-23   863.880      1
2017-11-05  2017-11-05   796.036      1
            2017-11-18   409.548      1

[4621 rows x 2 columns]

## Решаем задачу     

In [11]:
orders_merged['order_date'] - orders_merged['first_order']

0         0 days
1219    455 days
1398    461 days
3463   1054 days
4102   1191 days
          ...   
4161      0 days
4397      0 days
4992    823 days
4477      0 days
5003    538 days
Length: 5009, dtype: timedelta64[ns]

In [12]:
orders_merged['order_date'] - orders_merged['first_order'] <= '365 days'

0        True
1219    False
1398    False
3463    False
4102    False
        ...  
4161     True
4397     True
4992    False
4477     True
5003    False
Length: 5009, dtype: bool

In [13]:
year_1_filter = orders_merged['order_date'] - orders_merged['first_order'] <= '365 days'
year_1_orders = orders_merged[year_1_filter]
year_1_orders.head()

,id,order_date,ship_mode,customer_id,sales,first_order
0,100006,2014-09-07,Standard,DK-13375,377.970,2014-09-07
1,100090,2014-07-08,Standard,EB-13705,699.192,2014-07-08
369,129938,2014-12-15,Second,EB-13705,445.802,2014-07-08
1164,128125,2015-03-31,Standard,EB-13705,120.756,2014-07-08
2,100293,2014-03-14,Standard,NF-18475,91.056,2014-03-14


In [14]:
cohorts = year_1_orders.groupby(['first_order','order_date'])['sales'].agg(['sum', 'count'])
cohorts.head()

sum  count
first_order order_date                 
2014-01-03  2014-01-03    16.448      1
            2014-11-12   153.112      1
2014-01-04  2014-01-04   288.060      1
2014-01-05  2014-01-05    19.536      1
2014-01-06  2014-01-06  4407.100      3

In [15]:
cohorts.groupby('first_order').sum().mean()

sum      1949.850803
count       3.988789
dtype: float64